### SNAO/North Atlantic Jet Exploration ###

We want to find out what is going on with the North Atlantic Jet and the SNAO during JJA. First we will look at reanalyis to try and diagnose the trends. Then we will look at what is going on in the dcppA-hindcast data. For this, we will consider u250/u300/u850 and MSLP fields for the summer NAO. First using reanalysis data back to 1940.

In [ ]:
# Notebook magic
%matplotlib inline
# Import auto-reload
%load_ext autoreload
%autoreload 2

# Local imports
import os
import sys
import glob
import random

# Third-party imports
import numpy as np
import xarray as xr
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os.path
from pathlib import Path

# Import the cdsapi
import cdsapi


In [ ]:
# Import the local functions
from JJA_SNAO_jet_functions import *

# Import the dictionaries
sys.path.append('/home/users/benhutch/skill-maps/')
import dictionaries as dicts

In [ ]:
# Set up the cdsapi client
c = cdsapi.Client()

In [ ]:
# Set up the parameters
model_variable = "u850"
obs_variable = "u"
level = "850"
experiment = "dcppA-hindcast"
start_year_mod = 1960
end_year_mod = 2018
start_year_obs = 1940
end_year_obs = 2023

In [ ]:
# Set up the request dictionary for ERA5
request_dict_era5 = {
    'variable': 'u',
    'product_type': 'monthly_averaged_reanalysis',
    'year': [x for x in range(start_year_obs, end_year_obs)],
    'month': [x for x in range(6, 8 + 1)],
    'pressure_level': [850, 300, 250],
    'format': 'netcdf',
    'time': '00:00'
}

In [ ]:
# Set up the target directory
# ----------------------------
target_dir = "/gws/nopw/j04/canari/users/benhutch/ERA5/"
era5_target_filename = "era5-monthly-u850-300-250-1940-2023.nc"

# Check if the target directory exists
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Form the path
era5_target_path = os.path.join(target_dir, era5_target_filename)

# Print the path
print(era5_target_path)

# If the path does not exist, then download the data
if not os.path.exists(era5_target_path):
    c.retrieve(
        "reanalysis-era5-pressure-levels-monthly-means",
        request_dict_era5,
        era5_target_path
    )
else:
    print("The file already exists")
    print("Skipping download")
    print("using the existing file: ", era5_target_path)

In [ ]:
# Set up another request for psl back to 1940
# -------------------------------------------
request_dict_era5_psl = {
    'variable': 'mean_sea_level_pressure',
    'product_type': 'monthly_averaged_reanalysis',
    'year': [x for x in range(start_year_obs, end_year_obs)],
    'month': [x for x in range(6, 8 + 1)],
    'format': 'netcdf',
    'time': '00:00'
}

### Download psl for SNAO ###

Set this going at the end of the day

In [ ]:
# # Set up the target filename
# era5_target_filename_psl = "era5-monthly-psl-1940-2023.nc"

# # Check if the target directory exists
# if not os.path.exists(target_dir):
#     os.makedirs(target_dir)

# # Form the path
# era5_target_path_psl = os.path.join(target_dir, era5_target_filename_psl)

# # Print the path
# print(era5_target_path_psl)

# # If the path does not exist, then download the data
# if not os.path.exists(era5_target_path_psl):
#     c.retrieve(
#         "reanalysis-era5-single-levels-monthly-means",
#         request_dict_era5_psl,
#         era5_target_path_psl
#     )
# else:
#     print("The file already exists")
#     print("Skipping download")
#     print("using the existing file: ", era5_target_path_psl)

In [ ]:
# Calculate the climatology for the full period, U250
# -------------------------
climatology_era5_u250 = calculate_climatology(
    data_path=era5_target_path,
    variable="u",
    season="JJA",
    region=dicts.pacific_grid_plot_test,
    level=250
)

In [ ]:
climatology_era5_u250

In [ ]:
# Plot the climatology
# --------------------
# Assuming climatology_era5_u250 is an xarray DataArray
# First extract the data
climatology_era5_u250_data = climatology_era5_u250.u

# Then plot the data
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Orthographic(-80, 35), facecolor="gray"))

# Plot the data
climatology_era5_u250_data.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="coolwarm",
    robust=True,
    cbar_kwargs={
        "label": "U250 (m/s)"
    }
)

# Add coastlines
ax.coastlines()

In [ ]:
# Test the new function
# ---------------------
swj_index = diagnose_swj_index(
    data_path=era5_target_path,
    start_year=start_year_obs,
    end_year=end_year_obs,
)

In [ ]:
print(swj_index)

In [ ]:
fig, ax = plt.subplots()

# Add a label to the plot
swj_index.plot(ax=ax, label='SWJ Index')

# Include the climatology of the SWJ index as a horizontal line
climatology_swj_index = swj_index.mean()

# Add the climatology as a horizontal line
ax.axhline(climatology_swj_index[0], color="k", linestyle="--", label="Climatology")

# Label the axes
ax.set_xlabel("Year")
ax.set_ylabel("SWJ Index (latitude)")

# Include a rolling 10-year mean
swj_index.rolling(10, center=True).mean().plot(ax=ax, color="k", label="10-year rolling mean")

# Add a legend
ax.legend()

### SWJ index ###

Breaks down into three phases?

* Poleward - 1940-1980
* Equatorward - 1980-2010
* Poleward - 2010-onwards

In [ ]:
# Calculate the climatology for the early period
years = [x for x in range(1940, 1980)]

climatology_era5_u250_early = calculate_climatology(
    data_path=era5_target_path,
    variable="u",
    season="JJA",
    region=dicts.pacific_grid_plot_test,
    level=250,
    years=years
)

# Calculate the climatology for the late period
years = [x for x in range(1980, 2010)]

climatology_era5_u250_late = calculate_climatology(
    data_path=era5_target_path,
    variable="u",
    season="JJA",
    region=dicts.pacific_grid_plot_test,
    level=250,
    years=years
)

# Calculate the climatology for the recent period
years = [x for x in range(2010, 2023)]

climatology_era5_u250_recent = calculate_climatology(
    data_path=era5_target_path,
    variable="u",
    season="JJA",
    region=dicts.pacific_grid_plot_test,
    level=250,
    years=years
)


In [ ]:
# Calculate the deviation of these from the full climatology
climatology_era5_u250_early_deviation = climatology_era5_u250_early.u - climatology_era5_u250.u

climatology_era5_u250_late_deviation = climatology_era5_u250_late.u - climatology_era5_u250.u

climatology_era5_u250_recent_deviation = climatology_era5_u250_recent.u - climatology_era5_u250.u

In [ ]:
# Plot the deviation
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Orthographic(-80, 35), facecolor="gray"))

# Plot the data
climatology_era5_u250_early_deviation.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="coolwarm",
    robust=True,
    cbar_kwargs={
        "label": "U250 (m/s)"
    }
)

# Add coastlines
ax.coastlines()


In [ ]:
# Plot the deviation
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Orthographic(-80, 35), facecolor="gray"))

# Plot the data
climatology_era5_u250_late_deviation.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="coolwarm",
    robust=True,
    cbar_kwargs={
        "label": "U250 (m/s)"
    }
)

# Add coastlines
ax.coastlines()

In [ ]:
# Plot the deviation
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Orthographic(-80, 35), facecolor="gray"))

# Plot the data
climatology_era5_u250_recent_deviation.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="coolwarm",
    robust=True,
    cbar_kwargs={
        "label": "U250 (m/s)"
    }
)

# Add coastlines
ax.coastlines()